# Checking on the metrics for the counts head of the FOSL2 CUT&RUN models
*test set only (chr1)*

In [2]:
import os
os.chdir('/home/katie/bp_repo/reports/')

from util import import_profiles
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from IPython.display import display
import tqdm

import sys
sys.path.append('/home/katie/bp_repo/multitask_profile_model_SPI_GATA/')
import profile_performance
from profile_performance import *
from plotting_helper import *
from all_functions import *
from profile_models import ModelLoader

tqdm.tqdm_notebook()

plt.rcParams["figure.dpi"] = 130

/tmp/ipykernel_34084/782289464.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm.tqdm_notebook()


0it [00:00, ?it/s]

In [3]:
fosl2_cut_preds_dict = {}

### Single-task models

In [37]:
paths = ['/home/katie/bp_repo/reports/katie_notebooks/june2022_analysis/fosl2_cutnrun_preds/fosl2_cutnrun_mar6/',
         '/home/katie/bp_repo/reports/katie_notebooks/june2022_analysis/fosl2_cutnrun_preds/fosl2_cutnrun_jul5/',
         '/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/FOSL2_cut_model_']
path_ids = ['mar6', 'jul5', 'feb7']

keys = ['shared_chip_loader',
 'shared_cut_loader',
 'unique_chip_loader',
 'unique_cut_loader']

data_ids = ['true_counts', 'log_pred_counts', 'true_profs', 'log_pred_profs']

for i in range(3):
    path = paths[i]
    path_id = path_ids[i]
    preds_dict = {}
    
    # the surprisingly quick (few secs) actual preds loading
    for key in keys:
        file = h5py.File(f'{path}{key}.preds', 'r')
        coords = import_profiles(f'{path}{key}.preds')[2]
        preds_dict[key] = {'true_counts': file['predictions']['true_counts'], 
                           'log_pred_counts': file['predictions']['log_pred_counts'], 
                           'true_profs': file['predictions']['true_profs'],
                           'log_pred_profs': file['predictions']['log_pred_profs'], 
                           'coords': coords}
        
        # we need to subset to chr1, the test set
            # 1. take the coords, make into a DataFrame, subset to chr1, and get the indexes of the test coords
            # 2. use these indexes to subset pred_counts into the test profs
        df = pd.DataFrame(preds_dict[key]['coords']).reset_index()
        preds_dict[key]['coords_test'] = df[df.iloc[:,1] == 'chr1']
        preds_dict[key]['indexes_test'] = list(preds_dict[key]['coords_test']['index'])
        
        for data_id in data_ids:  # update all of the data to ONLY include test data
            preds_dict[key][data_id] = preds_dict[key][data_id][preds_dict[key]['indexes_test']]
    
    fosl2_cut_preds_dict[path_id] = preds_dict
    
# confirmed that all of the true_counts and true_profs are equal across models

Importing predictions:   0%|          | 0/54 [00:00<?, ?it/s]

Importing predictions:   0%|          | 0/51 [00:00<?, ?it/s]

Importing predictions:   0%|          | 0/14 [00:00<?, ?it/s]

Importing predictions:   0%|          | 0/41 [00:00<?, ?it/s]

Importing predictions:   0%|          | 0/54 [00:00<?, ?it/s]

...

Importing predictions:   0%|          | 0/41 [00:00<?, ?it/s]

Importing predictions:   0%|          | 0/54 [00:00<?, ?it/s]

Importing predictions:   0%|          | 0/51 [00:00<?, ?it/s]

Importing predictions:   0%|          | 0/14 [00:00<?, ?it/s]

Importing predictions:   0%|          | 0/41 [00:00<?, ?it/s]

In [101]:
pearson_df = pd.DataFrame(columns=path_ids + ['feb7_short', 'feb7_long'], index=keys)
spearman_df = pearson_df.copy()

In [102]:
path_ids = ['mar6', 'jul5', 'feb7']

keys = ['shared_chip_loader',
 'shared_cut_loader',
 'unique_chip_loader',
 'unique_cut_loader']

for path_id in path_ids:  # like mar6
    for key in keys:      # like shared_cut_loader
        preds_dict = fosl2_cut_preds_dict[path_id][key]
        true_counts, log_pred_counts, true_profs, log_pred_profs = \
        preds_dict['true_counts'], preds_dict['log_pred_counts'], preds_dict['true_profs'], preds_dict['log_pred_profs']
        metrics = profile_performance.compute_performance_metrics(
            true_profs, log_pred_profs, true_counts, log_pred_counts,
            prof_smooth_kernel_sigma=7, prof_smooth_kernel_width=81,
            print_updates=False
        )
        pearson_df.loc[key, path_id] = metrics['count_pearson']
        spearman_df.loc[key, path_id] = metrics['count_spearman']
        
# clean up lists into just numbers and round them
for path_id in path_ids:  # like mar6
    for key in keys:      # like shared_cut_loader
        pearson_df.loc[key, path_id] = np.round(pearson_df.loc[key, path_id][0], 2)
        spearman_df.loc[key, path_id] = np.round(spearman_df.loc[key, path_id][0], 2)

### Multi-task feb 7 model

In [ ]:
# load in preds for multi-task model
path = '/home/katie/bp_repo/reports/katie_notebooks/june2022_analysis/fosl2_cutnrun_preds/fosl2_cutnrun_120_150_feb7/'
data_ids = ['true_counts', 'log_pred_counts_short', 'log_pred_counts_long', 'true_profs', 'log_pred_profs_short', 'log_pred_profs_long']

preds_dict = {}
keys = ['shared_chip_loader',
 'shared_cut_loader',
 'unique_chip_loader',
 'unique_cut_loader']

# the surprisingly quick (few secs) actual preds loading
for key in keys:
    file = h5py.File(f'{path}{key}.preds', 'r')
    coords = import_profiles(f'{path}{key}.preds')[2]
    preds_dict[key] = {'true_counts': np.array(file['predictions']['true_counts']), 
                       'log_pred_counts_short': file['predictions']['log_pred_counts'][:,0:1,:],
                       'log_pred_counts_long': file['predictions']['log_pred_counts'][:,1:,:],
                       'true_profs': np.array(file['predictions']['true_profs']),
                       'log_pred_profs_short': file['predictions']['log_pred_profs'][:,0:1,:], 
                       'log_pred_profs_long': file['predictions']['log_pred_profs'][:,1:,:],
                       'coords': coords}


 # we need to subset to chr1, the test set
    # 1. take the coords, make into a DataFrame, subset to chr1, and get the indexes of the test coords
    # 2. use these indexes to subset pred_counts into the test profs
df = pd.DataFrame(preds_dict[key]['coords']).reset_index()
preds_dict[key]['coords_test'] = df[df.iloc[:,1] == 'chr1']
preds_dict[key]['indexes_test'] = list(preds_dict[key]['coords_test']['index'])

for data_id in data_ids:  # update all of the data to ONLY include test data
    preds_dict[key][data_id] = preds_dict[key][data_id][preds_dict[key]['indexes_test']]
    
fosl2_cut_preds_dict['feb7_multi'] = preds_dict

In [103]:
path_ids = ['feb7_short', 'feb7_long']

for path_id in path_ids:  # feb7_short or feb7_long
    for key in keys:      # like shared_cut_loader
        preds_dict = fosl2_cut_preds_dict['feb7_multi'][key]
        true_counts, log_pred_counts, true_profs, log_pred_profs = preds_dict['true_counts'], \
        preds_dict['log_pred_counts' + path_id[4:]], preds_dict['true_profs'], preds_dict['log_pred_profs' + path_id[4:]]
        metrics = profile_performance.compute_performance_metrics(
            true_profs, log_pred_profs, true_counts, log_pred_counts,
            prof_smooth_kernel_sigma=7, prof_smooth_kernel_width=81,
            print_updates=False
        )
        pearson_df.loc[key, path_id] = metrics['count_pearson']
        spearman_df.loc[key, path_id] = metrics['count_spearman']
        
# clean up lists into just numbers and round them
for path_id in path_ids:  # like mar6
    for key in keys:      # like shared_cut_loader
        pearson_df.loc[key, path_id] = np.round(pearson_df.loc[key, path_id][0], 2)
        spearman_df.loc[key, path_id] = np.round(spearman_df.loc[key, path_id][0], 2)

In [104]:
pearson_df

,mar6,jul5,feb7,feb7_short,feb7_long
shared_chip_loader,0.11,0.14,0.01,0.02,0.02
shared_cut_loader,0.18,0.22,0.01,0.02,0.02
unique_chip_loader,0.08,0.04,-0.02,-0.03,-0.09
unique_cut_loader,0.31,0.28,-0.0,-0.02,-0.12


In [105]:
spearman_df

,mar6,jul5,feb7,feb7_short,feb7_long
shared_chip_loader,0.08,0.11,0.03,0.04,-0.0
shared_cut_loader,0.17,0.2,0.03,0.05,0.0
unique_chip_loader,0.09,0.06,-0.05,-0.1,-0.1
unique_cut_loader,0.29,0.26,-0.01,-0.06,-0.12
